In [6]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer

from tools.classical import *
from tools.quantum import *

In [7]:

#* Qiskit Convention: indices run from TOP -> BOTTOM and resulting bitstrings are read from RIGHT -> LEFT
#* It's like a fallen tree, the LSB in circuit is the top one, and in the resulting bitstring then the right most one
num_qubits = 5
qr = QuantumRegister(num_qubits)
cr = ClassicalRegister(num_qubits)
circ = QuantumCircuit(qr, cr)
i_qiskit = 1
j_qiskit = 2

circ.x(qr[i_qiskit])
circ.x(qr[j_qiskit])
circ.y(qr[j_qiskit])
circ.measure(qr, cr)

tr_circ = transpile(circ, basis_gates=['cx', 'u3'], optimization_level=3)
simulator = Aer.get_backend('statevector_simulator')
shots = 1000
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

print(counts)

{'00010': 1000}


In [10]:

#* Qutip and my Numpy way of tensor product gives the same result as Qiskit -> THEY DON'T HAVE REVERSED ORDER

def tensor_product(arrays : list[np.ndarray]):
    """Returns the tensor product of the given arrays"""
    if len(arrays) == 1:
        return arrays[0]
    else:
        return np.kron(arrays[0], tensor_product(arrays[1:]))

X = np.array([[0, 1], [1, 0]]) # Pauli X
Y = np.array([[0, -1j], [1j, 0]]) # Pauli Y
I = np.eye(2)
op_list_qiskit_indices_XX = [I, X, X, I, I]
op_list_qiskit_indices_XX.reverse()
op_list = op_list_qiskit_indices_XX
opXX = tensor_product(op_list)

op_list_qiskit_indices_Y = [I, I, Y, I, I]
op_list_qiskit_indices_Y.reverse()
op_list = op_list_qiskit_indices_Y
opY = tensor_product(op_list)


# op_list_qiskit_indices = [qt.Qobj(I), qt.Qobj(X), qt.Qobj(X), qt.Qobj(I), qt.Qobj(I)]
# op_list_qiskit_indices.reverse()
# op_list = op_list_qiskit_indices
# op = qt.tensor(op_list).full()

# opXX = pad_term([qt.Qobj(X), qt.Qobj(X)], num_qubits, i_qiskit).full()
# opY = pad_term([qt.Qobj(Y)], num_qubits, j_qiskit).full()

op = opY @ opXX

In [11]:
op = Operator(op)
circ_np = QuantumCircuit(qr, cr)
circ_np.append(op, qr)
circ_np.measure(qr, cr)

tr_circ_np = transpile(circ_np, basis_gates=['cx', 'u3'], optimization_level=3)
simulator = Aer.get_backend('statevector_simulator')
shots = 1000
job = simulator.run(tr_circ_np, shots=shots)
counts = job.result().get_counts()
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
print(counts)


{'00010': 1000}


In [13]:
# np.random.seed(138)
# dist_qutip_qiskit = 0.
# while dist_qutip_qiskit < 1:
step_size = np.random.rand()
step_size = 1.1
print(f'RXX angle , {step_size}')
i = np.random.randint(0, num_qubits)
j = (i + 1) % num_qubits
print(f'Sites {i, j}')

#* Qutip
XX = pad_term([qt.sigmax(), qt.sigmax()], num_qubits, i)
eXX = expm(1j*step_size*XX.full())

# XX_byhand_reversed = qt.tensor([qt.qeye(2), qt.sigmax(), qt.sigmax()])
# eXX_byhand_reversed = expm(1j*step_size*XX_byhand_reversed.full())

print('Qutip')
print(eXX)
#* Qiskit
rxx_circ = QuantumCircuit(num_qubits)
rxx_circ.rxx(-2 * step_size, i, j)
rxx_op = Operator(rxx_circ)
print('Qiskit')
print(rxx_op)

dist_qutip_qiskit = np.linalg.norm(eXX - rxx_op.data)
# dist_qutip_qiskit_reversed = np.linalg.norm(eXX_byhand_reversed - rxx_op.data)
print(f'Distance between Qutip and Qiskit RXX {dist_qutip_qiskit}')
# print(f'Distance between Qutip and Qiskit RXX reversed {dist_qutip_qiskit_reversed}')


RXX angle , 1.1
Sites (2, 3)
Qutip
[[0.45359612+0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.45359612+0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.45359612+0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 ...
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.45359612+0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.45359612+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.45359612+0.j]]
Qiskit
Operator([[0.45359612+0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.45359612+0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.45359612+0.j, ...,
           0.        +0.j, 0.        +0.j, 0. 

### Register order

In [21]:
num_qubits_0 = 4
num_qubits_1 = 6
qr0 = QuantumRegister(num_qubits_0, name='A')
qr1 = QuantumRegister(num_qubits_1, name='B')
cr0 = ClassicalRegister(num_qubits_0)
cr1 = ClassicalRegister(num_qubits_1)

circ = QuantumCircuit(qr0, qr1, cr0, cr1)
circ.x(qr0[1])
circ.x(qr1[4])
print(circ)

circ.measure(qr0, cr0)
circ.measure(qr1, cr1)



tr_circ = transpile(circ, basis_gates=['cx', 'u3'], optimization_level=3)
simulator = Aer.get_backend('statevector_simulator')
shots = 1000
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
print(counts)


            ┌─┐                        
  A_0: ─────┤M├────────────────────────
       ┌───┐└╥┘                  ┌─┐   
  A_1: ┤ X ├─╫───────────────────┤M├───
       └───┘ ║ ┌─┐               └╥┘   
  A_2: ──────╫─┤M├────────────────╫────
             ║ └╥┘┌─┐             ║    
  A_3: ──────╫──╫─┤M├─────────────╫────
             ║  ║ └╥┘┌─┐          ║    
  B_0: ──────╫──╫──╫─┤M├──────────╫────
             ║  ║  ║ └╥┘┌─┐       ║    
  B_1: ──────╫──╫──╫──╫─┤M├───────╫────
             ║  ║  ║  ║ └╥┘┌─┐    ║    
  B_2: ──────╫──╫──╫──╫──╫─┤M├────╫────
             ║  ║  ║  ║  ║ └╥┘┌─┐ ║    
  B_3: ──────╫──╫──╫──╫──╫──╫─┤M├─╫────
       ┌───┐ ║  ║  ║  ║  ║  ║ └╥┘ ║ ┌─┐
  B_4: ┤ X ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├
       └┬─┬┘ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
  B_5: ─┤M├──╫──╫──╫──╫──╫──╫──╫──╫──╫─
        └╥┘  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
c16: 4/══╬═══╩══╩══╩══╬══╬══╬══╬══╩══╬═
         ║   0  2  3  ║  ║  ║  ║  1  ║ 
c17: 6/══╩════════════╩══╩══╩══╩═════╩═
         5            0  1  2  3     4 


/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
